This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
p = {'api_key': API_KEY, 'start_date': '2020-02-10', 'end_date': '2020-02-10'}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=p)
#model = r.json()
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2020-02-10',
  'end_date': '2020-02-10',
  'frequency': 'daily',
  'data': [['2020-02-10',
    114.0,
    121.1,
    107.0,
    109.4,
    None,
    449853.0,
    50064696.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#Task 1: Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
p = {'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'}
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params=p)


In [6]:
#Task 2: Convert the returned JSON object into a Python dictionary.
stock_dict = r.json()
type(stock_dict)

#It already is a dictionary, so hopefully this is all you had in mind

dict

In [7]:
#Task 3: Calculate what the highest and lowest opening prices were for the stock in this period.
high = 0
low = 100000000
for row in stock_dict['dataset_data']['data']:
    #print(row[1])
    #deal with null values for opening price
    if row[1] is not None:
        opening = row[1]
    else:
        opening = 0
    if opening > high:
        high = opening
    if (opening < low) and (opening > 0):
        low = opening
print('Highest Opening Stock Price: ' + str(high))
print('Lowest Opening Stock Price: ' + str(low))

Highest Opening Stock Price: 53.11
Lowest Opening Stock Price: 34.0


In [8]:
#Task 4: What was the largest change in any one day (based on High and Low price)?
change = 0
for row in stock_dict['dataset_data']['data']:
    temp_change = row[2] - row[3]
    if temp_change > change:
        change = temp_change
print('Largest change in one day: ' + str(change))

Largest change in one day: 2.8100000000000023


In [9]:
#Task 5: What was the largest change between any two days (based on Closing Price)?
#I'm assuming the change could be positive or negative
change2 = 0
last_closing = 0
positive = True
for row in stock_dict['dataset_data']['data']:
    if last_closing == 0:
        last_closing = row[4]
    else:
        temp_change = last_closing - row[4]
        if (abs(temp_change) > change2):
            change2 = temp_change
            positive = temp_change < 0
        last_closing = row[4]
if (positive):
    print('Largest change between two days: +' + str(change2))
else:
    print('Largest change between two days: -' + str(change2))

Largest change between two days: -1.4699999999999989


In [10]:
#Task 6: What was the average daily trading volume during this year?
total_vol = 0
trading_days = len(stock_dict['dataset_data']['data'])
for row in stock_dict['dataset_data']['data']:
    total_vol += row[6]
avg_vol = total_vol/trading_days
print('Average trading volume during 2017: ' + str(avg_vol))

Average trading volume during 2017: 89124.33725490196


In [11]:
#Task 7: (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#create list of all trading volumes
volumes = []
for row in stock_dict['dataset_data']['data']:
    volumes.append(row[6])

#sort trading volumes
volumes.sort()

#location of the median is the midpoint of the list
median_loc = len(volumes)/2

#if list has an odd number of items, median_loc will end with .5, and the right location is median_loc - .5 
if (median_loc // 1 > 0):
    median = volumes[int(median_loc - 0.5)]
else:
#otherwise list has an even number of items, and median is the average of the middle two items
    median = (volumes[int(median_loc - 1)] + volumes[int(median_loc)])/2
    
print('Median trading volume during 2017: ' + str(median))

Median trading volume during 2017: 76286.0
